In [1]:
from pkg_ML_V1 import *


Ready!


In [ ]:


# Specify the directory where the CSV files are located
folder_path = 'Generated_Data'

# Initialize a dictionary to store the parameters of each file
file_parameters = {}

# List all files in the directory
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Attempt to match the filename pattern
        match = re.match(r"Qubits_(\d+)_Lines_(\d+)_TS_(\d+)_Shots_(\d+)_MeanDecay_(\d+)", filename)
        if match:
            # Extract parameters from the filename
            qubits, lines, ts, shots, meandecay = match.groups()
            
            # Convert strings to integers
            qubits = int(qubits)
            lines = int(lines)
            ts = int(ts)
            shots = int(shots)
            meandecay = int(meandecay)
            
            # Store parameters in the dictionary
            file_parameters[filename] = {
                'Qubits': qubits,
                'Lines': lines,
                'TS': ts,
                'Shots': shots,
                'MeanDecay': meandecay
            }
            
            # Load the CSV file into a DataFrame (if needed)
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            
            # Here you can process the DataFrame as needed
            # ...

# Now file_parameters dictionary contains all the extracted information
print(file_parameters)
